In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import fix_yahoo_finance as yf
import arch
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
yf.pdr_override()

ImportError: cannot import name 'StringIO' from 'pandas.compat' (/anaconda3/lib/python3.7/site-packages/pandas/compat/__init__.py)

In [4]:
# Volatility prediction

class stock_vol:
    def __init__(self, tk, start, end):
          self.tk = tk
          self.start = start
          self.end = end
          all_data = pdr.get_data_yahoo(self.tk, start=self.start, end=self.end)
          self.stock_data = pd.DataFrame(all_data['Adj Close'], columns=["Adj Close"])
          self.stock_data["log"] = np.log(self.stock_data)-np.log(self.stock_data.shift(1))
    def garch_sigma(self):
          model = arch.arch_model(self.stock_data["log"].dropna(), mean='Zero', vol='GARCH', p=1, q=1)
          model_fit = model.fit()
          forecast = model_fit.forecast(horizon=1)
          var = forecast.variance.iloc[-1]
          sigma = float(np.sqrt(var))
          return sigma

In [5]:
def mean_sigma(self):
   st = self.stock_data["log"].dropna().ewm(span=252).std()
   sigma = st.iloc[-1]
   return sigma

In [6]:
# Building pricing tree

class euro_option(stockoption):
   '''
   calculate required preliminary parameters:
   u = factor change of upstate
   d = factor change of downstate
   qu = risk free upstate probability
   qd = risk free downstate probability
   M = number of nodes
   '''
   def __int_prms__(self):
      self.M = self.N + 1 
      self.u = math.exp(self.sigma*math.sqrt(self.dt))
      self.d = 1./self.u
      self.qu = (math.exp((self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
      self.qd = 1-self.qu
      
   def stocktree(self):
      stocktree = np.zeros([self.M, self.M])
      for i in range(self.M):
         for j in range(self.M):
            stocktree[j, i] = self.S0*(self.u**(i-j))*(self.d**j)
      return stocktree
def option_price(self, stocktree):
      option = np.zeros([self.M, self.M])
      if self.is_call:
         option[:, self.M-1] = np.maximum(np.zeros(self.M), (stocktree[:, self.N] - self.K))
      else:
         option[:, self.M-1] = np.maximum(np.zeros(self.M), (self.K - stocktree[:, self.N]))
      return option
def optpricetree(self, option):
      for i in np.arange(self.M-2, -1, -1):
         for j in range(0, i+1):
            option[j, i] = math.exp(-self.r*self.dt) * (self.qu*option[j, i+1]+self.qd*option[j+1, i+1])
      return option
def begin_tree(self):
      stocktree = self.stocktree()
      payoff = self.option_price(stocktree)
      return self.optpricetree(payoff)
def price(self):
      self.__int_prms__()
      self.stocktree()
      payoff = self.begin_tree()
      return payoff[0, 0]

NameError: name 'stockoption' is not defined